# Data Model (Object Model)

## 1. special method (dunder method)

> special method를 사용하면 내가 정의한 클래스를 build-in object(내장형 객체)로 만들어 build-in function(내장 함수)를 사용할 수 있다. (Pythonic!)


* usage: not for user (ex. `obj.__len__()`) but for python interpretor (ex. `len(obj)`) to read.   
    즉, 만약 내가 만든 class에 special method가 구현되어있다면 (ex. `__len__()`), python interpretor가 special method를 호출한다. 따라서 사용자는 special method자체를 호출하지 않고 내장 함수 (ex. `len()`)를 사용하듯이 호출하면 된다. 
    
* tips: `__init__` 을 제외하고는 특별 메소드를 구현하는 것보다 내장 함수(`len()`, `str()`, `iter()`, ...)를 사용하는 것이 효율적임. 또한 사용자 정의 메소드나 속성을 만들 때 dunder(`__myfunc__()`, `self.__myattr__`)의 사용은 특별 메소드와 헷갈릴 수 있기 때문에 지양한다. 
    
* types
    ```python
    __init__
    __iter__
    __str__
    ```

* special method를 이용하여 sequence object 만들기
    * special method
    
    ```python
    __getitem__()
    __len__()
    ```
    
    * 표준 라이브러리 함수를 사용 가능   
        ex: len(), list[i], random.choice(list), ...


* python built-in container datatypes : __list__, __dict__, __set__, __tuple__
* python library container datatypes (__iterable object__): [source](https://data-flair.training/blogs/python-sequence/)
    * __collections__
        * def : no-deterministic ordering
        * types: set, dict
    * __sequence__ 
        * def : deterministic ordering
        * types: list, tuple, string, range objects, bytes array(mutable), bytes sequence(immutable)

## [additional] mutable object vs immutable object

* definition
    * immutable : 변수에 값을 한 번 지정한 후에는 값 변경이 불가능. 만약 변수의 값을 변경하면 변수의 주소값(id)가 달라짐. 즉, 다른 변수가 됨.
    * mutable : 변수에 값을 지정한 후 값 변경이 가능. 주소값도 동일.
* types
    * immutable: string, tuple, bytes sequence
    * mutable: list, range obejects, bytes array, set, dict
    

#### 1-1. card

In [72]:
import collections

Card = collections.namedtuple('Card', ['rank','suit']) #(1)

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') # class attribute?
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks] # double list comprehension
        
    
    def __len__(self): #(2-1) 
        return len(self._cards)
    
    
    def __getitem__(self, position): #(2-2) 
        return self._cards[position]
        

In [67]:
#(1) method없이 attribute로만 구현된 class 
beer_card = Card("7","diamonds")
beer_card.rank

'7'

In [73]:
#(2) special(dunder) method overriding 
deck = FrenchDeck()

print(len(deck)) #(2-1)
print(deck[0]) #(2-2) getitem 메소드를 가지면 sequence object가 되어 slicing이 가능해진다. 
print(deck[51])



52
Card(rank='2', suit='spades')
Card(rank='A', suit='hearts')


In [74]:
#(2-2) __getitem__ -> sequence object 
from random import choice # sequence object에서 임의의 값을 가져오는 메소드
print(choice(deck))

print(deck[12::13]) # slicing

for card in reversed(deck): # for loop
    print(card)
    break



Card(rank='A', suit='clubs')
[Card(rank='A', suit='spades'), Card(rank='A', suit='diamonds'), Card(rank='A', suit='clubs'), Card(rank='A', suit='hearts')]
Card(rank='A', suit='hearts')


In [75]:
# (2-2) sequence -> in 가능
# __contains__ method -> in 연산 가능
Card(rank='A', suit='clubs') in deck

True

In [89]:
# (2-2) sequence -> for loop 가능 
## priority로 sorting
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_values = FrenchDeck.ranks.index(card.rank)
    return rank_values * len(suit_values) + suit_values[card.suit] # rank가 우선 순위, rank가 같다면 suit비교

for card in sorted(deck, key = spades_high):
    print(card)
    break

Card(rank='2', suit='clubs')


#### 1-2. vector

In [112]:
from math import hypot # Euclidean norm

class Vector:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    
    def __repr__(self): #(1)
        return f"Vector({self.x},{self.y})"
    
    
    def __abs__(self): 
        return hypot(self.x, self.y)
    
    
    def __bool__(self): #(2)
        return bool(abs(self))
    
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x,y)
    
    
    def __mul__(self, scalar):
        return Vector(self.x*scalar, self.y*scalar)
    

In [114]:
# (1) 객체를 문자열(class 호출하는 모습과 동일)로 표현하는 특별 메소드 __repr__()
Vector(2,3)

Vector(2,3)

In [122]:
# (2) 파이썬 인터프리터는 오브젝트에 __bool__()이 정의되지 않는다면 __len__()을 호출 ex. list object
print(bool(Vector()))

obj = Vector()
if obj:
    print(f"truthy {obj}")
else:
    print(f"falsy {obj}")

False
falsy Vector(0,0)
